In [24]:
import requests
import json
import pandas as pd
import time
import datetime
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras

In [57]:
code = # enter key from FitBit here
my_headers = {'Authorization' :  'Bearer ' + code}
response = requests.get('https://api.fitbit.com/1.2/user/-/sleep/list.json?beforeDate=2022-05-06&sort=desc&offset=0&limit=100', headers=my_headers)
print(response)

<Response [200]>


In [8]:
data = response.json()

In [9]:
print(data)

{'pagination': {'beforeDate': '2022-05-06', 'limit': 100, 'next': 'https://api.fitbit.com/1.2/user/-/sleep/list.json?offset=100&limit=100&sort=desc&beforeDate=2022-05-06', 'offset': 0, 'previous': '', 'sort': 'desc'}, 'sleep': [{'dateOfSleep': '2022-05-05', 'duration': 33000000, 'efficiency': 93, 'endTime': '2022-05-05T06:57:00.000', 'infoCode': 0, 'isMainSleep': True, 'levels': {'data': [{'dateTime': '2022-05-04T21:47:00.000', 'level': 'wake', 'seconds': 420}, {'dateTime': '2022-05-04T21:54:00.000', 'level': 'light', 'seconds': 1170}, {'dateTime': '2022-05-04T22:13:30.000', 'level': 'deep', 'seconds': 1530}, {'dateTime': '2022-05-04T22:39:00.000', 'level': 'light', 'seconds': 150}, {'dateTime': '2022-05-04T22:41:30.000', 'level': 'wake', 'seconds': 600}, {'dateTime': '2022-05-04T22:51:30.000', 'level': 'light', 'seconds': 150}, {'dateTime': '2022-05-04T22:54:00.000', 'level': 'rem', 'seconds': 2160}, {'dateTime': '2022-05-04T23:30:00.000', 'level': 'light', 'seconds': 1890}, {'dateTim

In [10]:
with open('sleep_repo', 'w') as f:
  json.dump(data, f, ensure_ascii=False)

In [11]:
sleep = pd.json_normalize(data, sep=',')

In [12]:
sleep2 = sleep['sleep'][0]

In [13]:
print(sleep2)

[{'dateOfSleep': '2022-05-05', 'duration': 33000000, 'efficiency': 93, 'endTime': '2022-05-05T06:57:00.000', 'infoCode': 0, 'isMainSleep': True, 'levels': {'data': [{'dateTime': '2022-05-04T21:47:00.000', 'level': 'wake', 'seconds': 420}, {'dateTime': '2022-05-04T21:54:00.000', 'level': 'light', 'seconds': 1170}, {'dateTime': '2022-05-04T22:13:30.000', 'level': 'deep', 'seconds': 1530}, {'dateTime': '2022-05-04T22:39:00.000', 'level': 'light', 'seconds': 150}, {'dateTime': '2022-05-04T22:41:30.000', 'level': 'wake', 'seconds': 600}, {'dateTime': '2022-05-04T22:51:30.000', 'level': 'light', 'seconds': 150}, {'dateTime': '2022-05-04T22:54:00.000', 'level': 'rem', 'seconds': 2160}, {'dateTime': '2022-05-04T23:30:00.000', 'level': 'light', 'seconds': 1890}, {'dateTime': '2022-05-05T00:01:30.000', 'level': 'deep', 'seconds': 3090}, {'dateTime': '2022-05-05T00:53:00.000', 'level': 'light', 'seconds': 60}, {'dateTime': '2022-05-05T00:54:00.000', 'level': 'rem', 'seconds': 1020}, {'dateTime': 

In [58]:
# reported in milliseconds converted to minutes
print(sleep['sleep'][0][0]['duration']/60000)

550.0


Creating the data frame that all of the parsed data will be added to

In [14]:
sleep_df = pd.DataFrame()

In [15]:
print(sleep2[79])

{'dateOfSleep': '2019-03-09', 'duration': 5340000, 'efficiency': 100, 'endTime': '2019-03-09T00:26:30.000', 'infoCode': 2, 'isMainSleep': True, 'levels': {'data': [{'dateTime': '2019-03-08T22:57:30.000', 'level': 'asleep', 'seconds': 5340}], 'summary': {'asleep': {'count': 0, 'minutes': 89}, 'awake': {'count': 0, 'minutes': 0}, 'restless': {'count': 0, 'minutes': 0}}}, 'logId': 21523949852, 'logType': 'auto_detected', 'minutesAfterWakeup': 0, 'minutesAsleep': 89, 'minutesAwake': 0, 'minutesToFallAsleep': 0, 'startTime': '2019-03-08T22:57:30.000', 'timeInBed': 89, 'type': 'classic'}


# Appending data to dataframe

This function adds the data from the json file to the dataframe

In [50]:
def append_sleep():
    sleep_df['dateOfSleep'] = [0]*len(sleep2)
    sleep_df['duration'] = [0]*len(sleep2)
    sleep_df['isMainSleep'] = [0]*len(sleep2)
    sleep_df['deepSleep'] = [0]*len(sleep2)
    sleep_df['lightSleep'] = [0]*len(sleep2)
    sleep_df['remSleep'] = [0]*len(sleep2)
    sleep_df['wakeSleep'] = [0]*len(sleep2)
    sleep_df['asleepSleep']  = [0]*len(sleep2)
    sleep_df['awakeSleep'] = [0]*len(sleep2)
    sleep_df['restlessSleep'] = [0]*len(sleep2)
    for i in range(0,len(sleep2)):
        if sleep2[i]['levels']['summary'].get('asleep') is None:
            sleep_df['dateOfSleep'][i] = (datetime.datetime.strptime((sleep2[i]['dateOfSleep']), '%Y-%m-%d').date())
            sleep_df['duration'][i] =pd.to_numeric((sleep2[i]['duration']/60000))
            sleep_df['isMainSleep'][i] =(sleep2[i]['isMainSleep'])
            sleep_df['deepSleep'][i] =(sleep2[i]['levels']['summary']['deep']['minutes'])
            sleep_df['lightSleep'][i] =(sleep2[i]['levels']['summary']['light']['minutes'])
            sleep_df['remSleep'][i] =(sleep2[i]['levels']['summary']['rem']['minutes'])
            sleep_df['wakeSleep'][i] =(sleep2[i]['levels']['summary']['wake']['minutes'])
        else:
            sleep_df['dateOfSleep'][i] = (datetime.datetime.strptime((sleep2[i]['dateOfSleep']), '%Y-%m-%d').date())
            sleep_df['duration'][i] =pd.to_numeric((sleep2[i]['duration']/60000))
            sleep_df['isMainSleep'][i] =(sleep2[i]['isMainSleep'])
            sleep_df['asleepSleep'][i] =(sleep2[i]['levels']['summary']['asleep']['minutes'])
            sleep_df['awakeSleep'][i] =(sleep2[i]['levels']['summary']['awake']['minutes'])
            sleep_df['restlessSleep'][i] =(sleep2[i]['levels']['summary']['restless']['minutes'])
            
    return 0

In [51]:
append_sleep()

C:\Users\sbsch\AppData\Local\Temp/ipykernel_13388/1827531475.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df['duration'][i] =pd.to_numeric((sleep2[i]['duration']/60000))
C:\Users\sbsch\AppData\Local\Temp/ipykernel_13388/1827531475.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df['isMainSleep'][i] =(sleep2[i]['isMainSleep'])
C:\Users\sbsch\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

0

Saving the data to csv

In [52]:
sleep_df.to_csv('D:\data.csv')

In [53]:
print(sleep_df)

   dateOfSleep  duration isMainSleep  deepSleep  lightSleep  remSleep  \
0   2022-05-05       550        True         80         289       111   
1   2022-05-04       527        True         67         324        46   
2   2022-05-03       137       False          0           0         0   
3   2022-05-03       530        True         87         322        51   
4   2022-05-02       431        True         50         291        46   
..         ...       ...         ...        ...         ...       ...   
95  2019-02-03       350        True         18         259        37   
96  2019-02-02       473        True         11         353        61   
97  2019-01-31       395        True         37         235        71   
98  2019-01-30       509        True         72         279       104   
99  2019-01-28       517        True         69         297        73   

    wakeSleep  asleepSleep  awakeSleep  restlessSleep  
0          70            0           0              0  
1          

# Adding the data to the Postgres database

In [55]:
conn = psycopg2.connect(database="sleep_study",
                        user='postgres', password= #database password, 
                        host='127.0.0.1', port='5432'
)
  
conn.autocommit = True
cursor = conn.cursor()
  
  

  
sql2 = '''COPY sleep_data(dateOfSleep,duration,isMainSleep,deepSleep,lightSleep, \
remSleep,wakeSleep,asleepSleep,awakeSleep,restlessSleep)
FROM 'D:\\data.csv'
DELIMITER ','
CSV HEADER;'''
  
cursor.execute(sql2)
  

conn.commit()
conn.close()